In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import NearMiss
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf

In [2]:
df_reg = pd.read_csv('../data/Lab3_Mrdak_MO-221.csv')
df_reg.head()

,Unnamed: 0,date,appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,...,RH_8,T9,RH_9,T_out,press_mm_hg,RH_out,windspeed,visibility,tdewpoint,weekday
0,0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,...,48.900000,17.033333,45.53,6.60,733.5,92.0,7.000000,63.000000,5.3,Monday
1,1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,...,48.863333,17.066667,45.56,6.48,733.6,92.0,6.666667,59.166667,5.2,Monday
2,2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,...,48.730000,17.000000,45.50,6.37,733.7,92.0,6.333333,55.333333,5.1,Monday
3,3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,...,48.590000,17.000000,45.40,6.25,733.8,92.0,6.000000,51.500000,5.0,Monday
4,4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,...,48.590000,17.000000,45.40,6.13,733.9,92.0,5.666667,47.666667,4.9,Monday


In [3]:
df_class = pd.read_csv('../data/Lab4_Mrdak_MO-221.csv')
df_class.head()

,Unnamed: 0,name,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,absolute_magnitude,hazardous
0,0,(2011 GZ2),0.016016,0.035813,56014.078517,1.024333e+06,26.1,0
1,1,(2020 HT6),0.030518,0.068240,7864.348060,3.268186e+07,24.7,0
2,2,(2016 ED156),0.055533,0.124177,55257.544508,6.538636e+07,23.4,0
3,3,(2013 FD8),0.019256,0.043057,41531.404722,1.260796e+07,25.7,0
4,4,(2016 CW31),0.139494,0.311918,67639.394481,7.130590e+07,21.4,0


## Удаление ненужных столбцов

In [4]:
df_reg = df_reg.drop(['Unnamed: 0', 'date', 'weekday'], axis=1)
df_reg.head()

,appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T8,RH_8,T9,RH_9,T_out,press_mm_hg,RH_out,windspeed,visibility,tdewpoint
0,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,...,18.2,48.900000,17.033333,45.53,6.60,733.5,92.0,7.000000,63.000000,5.3
1,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,...,18.2,48.863333,17.066667,45.56,6.48,733.6,92.0,6.666667,59.166667,5.2
2,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,...,18.2,48.730000,17.000000,45.50,6.37,733.7,92.0,6.333333,55.333333,5.1
3,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,...,18.1,48.590000,17.000000,45.40,6.25,733.8,92.0,6.000000,51.500000,5.0
4,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,...,18.1,48.590000,17.000000,45.40,6.13,733.9,92.0,5.666667,47.666667,4.9


In [5]:
df_class = df_class.drop(['Unnamed: 0', 'name'], axis=1)
df_class.head()

,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,absolute_magnitude,hazardous
0,0.016016,0.035813,56014.078517,1.024333e+06,26.1,0
1,0.030518,0.068240,7864.348060,3.268186e+07,24.7,0
2,0.055533,0.124177,55257.544508,6.538636e+07,23.4,0
3,0.019256,0.043057,41531.404722,1.260796e+07,25.7,0
4,0.139494,0.311918,67639.394481,7.130590e+07,21.4,0


## Выделение целевого признака для обоих датасетов

In [6]:
y_reg = df_reg["appliances"]
X_reg = df_reg.drop(["appliances"], axis=1)

In [7]:
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, stratify = y_reg)

In [8]:
y_class = df_class["hazardous"]
X_class = df_class.drop(["hazardous"], axis=1)

In [9]:
nm = NearMiss()
X_class, y_class = nm.fit_resample(X_class, y_class.ravel())

In [10]:
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_class, y_class, test_size=0.2, stratify = y_class)

## Масштабирование

In [11]:
sc_reg = StandardScaler()
X_train_reg = sc_reg.fit_transform(X_train_reg)
X_test_reg = sc_reg.transform(X_test_reg)

In [12]:
sc_class = StandardScaler()
X_train_class = sc_class.fit_transform(X_train_class)
X_test_class = sc_class.transform(X_test_class)

## Создание вспомогательных функций

In [13]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from math import sqrt
def metrics(name, y_test, y_pred):
    print(name)
    print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
    print(f'MSE: {mean_squared_error(y_test, y_pred)}')
    print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
    print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')
    print(f'R^2: {r2_score(y_test, y_pred)}')

In [14]:
from sklearn.metrics import roc_auc_score
def auc_roc(X_test, y_test, model):
    y_scores = model.predict_proba(X_test)   
    print(roc_auc_score(y_test, y_scores[:,1]))

## Построение моделей

### Регрессия

In [15]:
model_regression = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(25,)),
        tf.keras.layers.Dense(32, activation="linear"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(1, activation="linear"),
    ]
)

In [16]:
model_regression.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                1664      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                1056      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 4833 (18.88 KB)
Trainable params: 4833 (18

In [17]:
model_regression.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="mse")

In [18]:
model_regression.fit(X_train_reg, y_train_reg, epochs=150, verbose = None)

In [19]:
metrics("Регрессия", y_test_reg, model_regression.predict(X_test_reg))

110/110 [==============================] - 0s 844us/step
Регрессия
MAE: 12.209243425455961
MSE: 320.51898808928075
RMSE: 17.90304410119354
MAPE: 0.2021394506264495
R^2: 0.6046918050938588


### Классификация

In [20]:
model_classification = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(5,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(2, activation="softmax"),
    ]
)

In [21]:
model_classification.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="sparse_categorical_crossentropy")

In [22]:
model_classification.fit(X_train_class, y_train_class, epochs=50)

Epoch 1/50
322/322 [==============================] - 1s 1ms/step - loss: 0.2818
Epoch 2/50
322/322 [==============================] - 0s 1ms/step - loss: 0.2683
Epoch 3/50
322/322 [==============================] - 0s 1ms/step - loss: 0.2644
Epoch 4/50
322/322 [==============================] - 0s 1ms/step - loss: 0.2659
Epoch 5/50
322/322 [==============================] - 0s 1ms/step - loss: 0.2625
Epoch 6/50
322/322 [==============================] - 0s 1ms/step - loss: 0.2602
Epoch 7/50
322/322 [==============================] - 0s 1ms/step - loss: 0.2608
Epoch 8/50
322/322 [==============================] - 0s 1ms/step - loss: 0.2590
Epoch 9/50
322/322 [==============================] - 0s 1ms/step - loss: 0.2611
Epoch 10/50
322/322 [==============================] - 0s 1ms/step - loss: 0.2582
Epoch 11/50
322/322 [==============================] - 0s 1ms/step - loss: 0.2603
Epoch 12/50
322/322 [==============================] - 0s 1ms/step - loss: 0.2586
Epoch 13/50
322/322 [====

In [23]:
y_pred = [np.argmax(pred) for pred in model_classification.predict(X_test_class)]

81/81 [==============================] - 0s 800us/step


In [24]:
print(classification_report(y_test_class, y_pred))
print(confusion_matrix(y_test_class, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.81      0.89      1288
           1       0.84      1.00      0.91      1288

    accuracy                           0.90      2576
   macro avg       0.92      0.90      0.90      2576
weighted avg       0.92      0.90      0.90      2576

[[1044  244]
 [   4 1284]]


# Реализация MLP

Задание №2. Разработать многослойный персептрон (MLP), с помощью которого можно решать задачи регрессии и классификации. Предусмотреть возможность использования таких функции активации, как sigmoid, tanh и relu; также предусмотреть возможность указать, сколько слоев нужно, сколько на каждом из них нейронов и какую функцию активации должен иметь слой. Реализовать обучение MLP методом обратного распространения ошибки; самостоятельно найти производные функций sigmoid, tanh и relu; реализовать классический градиентный спуск с возможностью указания шага.

In [25]:
#input_layer = [64, "relu", 25]
#hidden_layers = [[64, "relu"], [32, "relu"], [16, "relu"]]
#output_layer = [1, "relu"]

In [26]:
class MLP:
    def __init__(self, input_size, hidden_layers):
        self.input_size = input_size
        self.hidden_layers = hidden_layers
        self.weights = []
        self.activation = []
        self.der_activation = []
        self.activation.append(self.relu)
        self.der_activation.append(self.relu)
        self.weights.append(np.random.randn(hidden_layers[0][0], input_size))
        for i in range (len(hidden_layers)-1):
            weights = []
            activation = []
            der_activation = []
            if hidden_layers[i][1] == "sigmoid":
                self.activation.append(self.sigmoid)
                self.der_activation.append(self.der_sigmoid)
            elif hidden_layers[i][1] == "tanh":
                self.activation.append(self.tanh)
                self.der_activation.append(self.der_tanh)
            else:
                self.activation.append(self.relu)
                self.der_activation.append(self.der_relu)

            matrix = np.random.randn(hidden_layers[i+1][0], hidden_layers[i][0])
            self.weights.append(matrix)
        
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    def relu(self, x):
        return np.maximum(0, x)
    def tanh(self, x):
        return np.tanh(x)
        
    def der_sigmoid(self, x):
        return x*(1-x)
    def der_relu(self, x):
        return 1 * (x>0)
    def der_tanh(self, x):
        return 1-x**2

    def mse(self, real, pred):
        return np.mean((real-pred)**2)

    def forward(self, x):
        self.act_values = [x]
        self.values = []
        for i in range(len(self.hidden_layers)):
            value = np.dot(self.weights[i], x)
            x = self.activation[i](value)
            self.values.append(np.matrix(value))
            self.act_values.append(np.matrix(x))
        print(self.act_values)
        return x
        
    def fit(self, X, y, l_rate, ep, loss_func):
        self.l_rate = l_rate
        if loss_func == "MSE":
            self.loss_func = self.mse
        for i in range (ep):
            l =[]
            for j in range(len(X)):
                out = self.forward(X[j].T)
                l.append(self.mse(y[j], out))
                self.backprop(np.matrix(X[j].T), np.matrix(y[j].T))
            print("Эпоха", i, " loss", loss)
            


    def backprop(self, x, y):
        error = self.act_values[-1] - y
        
        for i in range(len(self.hidden_layers) - 1, -1, -1):
            value = self.values[i]
            der = self.der_activation[i](value)
            dw = np.dot(error, self.act_values[i].T)
            self.weights[i] -= self.l_rate * dw
            error = np.dot(self.weights[i].T, error) * der
            
            
    def predict(self, X):
        return self.forward(X)
    

In [27]:
m = MLP(25, [[2, "relu"], [2, "relu"], [2, "relu"],[1, "relu"]])

In [28]:
m.fit(X_train_reg[:5], np.ravel(y_train_reg[:5]), ep=5, l_rate = 0.05, loss_func = "mse")

[array([-0.4582467 , -1.44481987, -0.52116753, -1.84562763,  0.39396062,
       -1.14187255, -0.6513495 , -0.59062035, -0.67856123, -1.04892827,
       -0.24249001, -0.43563082, -0.0905527 , -0.99541965, -0.62961516,
       -1.36722157, -0.70911756, -0.90476794, -0.13641009, -0.47282079,
       -1.45131005, -0.96503098,  1.64526378,  0.13577078, -1.15464678]), matrix([[0.        , 6.14750172]]), matrix([[2.68492767, 0.        ]]), matrix([[2.23470608, 0.        ]]), matrix([[0.]])]


ValueError: shapes (1,1) and (2,1) not aligned: 1 (dim 1) != 2 (dim 0)

In [29]:
import pickle

In [30]:
with open("DNNClass.pkl", "wb") as f:
    pickle.dump(model_classification, f)

In [32]:
model_classification.save('DNNClass.h5')

C:\Users\79237\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
